# Neighbouring Stations

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

path = 'mypath'

In [ ]:
df_metadata_train = pd.read_csv(path + 'df_metadata_train.csv', index_col=0)
df_metadata_train.index = df_metadata_train.index.astype(str)

df_metadata_test = pd.read_csv(path + 'df_metadata_test.csv', index_col=0)
df_metadata_test.index = df_metadata_test.index.astype(str)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
xr_train=xr.open_dataset(path+'xr_train_timestamped.netcdf')
xr_test=xr.open_dataset(path+'xr_test_timestamped.netcdf')

In [ ]:
import geopy.distance
import pandas as pd

def get_distances(coords, df_metadata_train):

  distances = np.zeros(df_metadata_train.shape[0])

  for i, ss in enumerate(df_metadata_train.index.values):

      lat_train = df_metadata_train.loc[ss]['latitude_rounded']
      lon_train = df_metadata_train.loc[ss]['longitude_rounded']

      coords_train = (lat_train, lon_train)

      distances[i] = geopy.distance.distance(coords, coords_train).km

  return distances


def df_of_neighbours(df_metadata, df_metadata_ref):
  print("Generating df of distances")
  print('Progress (out of %d): ' %(df_metadata.shape[0]//50), end='')

  res = np.zeros((df_metadata_ref.shape[0], df_metadata.shape[0]))

  for i, ss in enumerate(df_metadata.index.values):
    if i % 50 == 49: print('#', end='')
    lat = df_metadata.loc[ss]['latitude_rounded']
    lon = df_metadata.loc[ss]['longitude_rounded']
    coords = (lat, lon)

    res[:, i] = get_distances(coords, df_metadata_ref)

  print('  Done')

  return pd.DataFrame(data=res, index=df_metadata_ref.index.values, columns=df_metadata.index.values)

In [ ]:
df_distances_train = df_of_neighbours(df_metadata_train, df_metadata_train)

Generating df of distances
Progress (out of 13): #############  Done


In [ ]:
df_distances_test = df_of_neighbours(df_metadata_test, df_metadata_test)

Generating df of distances
Progress (out of 5): #####  Done


In [ ]:
df_distances_test.to_csv(path+'df_distances_test.csv')
df_distances_train.to_csv(path+'df_distances_train.csv')

In [ ]:
df_distances_test = pd.read_csv(path+'df_distances_test.csv', index_col=0)
df_distances_train = pd.read_csv(path+'df_distances_train.csv', index_col=0)
df_distances_test.index = df_distances_test.index.astype(str)
df_distances_train.index = df_distances_train.index.astype(str)

In [ ]:
from torch.utils.data import Dataset


def get_seq_length(past_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 1, 'hour': 0},
                   future_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 1},
                   max_len = 133992):
  look_up = (past_dict['hour'] * 4 + past_dict['day'] * 24 * 4
            + past_dict['month'] *30 * 24 * 4 + past_dict['year'] * 12 * 30 * 24 * 4)
  look_ahead = (future_dict['hour'] * 4 + future_dict['day'] * 24 * 4
                + future_dict['month'] *30 * 24 * 4 + future_dict['year'] * 12 * 30 * 24 * 4)

  if look_up+look_ahead > max_len:
    print('Not enough data to accomodate requested sequence lengths, returning 1 and 1')
    look_up, look_ahead = 1, 1

  return look_up, look_ahead


class PVDataset(Dataset):
  def __init__(self, X, df_distances, look_up=1, look_ahead=1,
               cut_off_km=64, n_neighbours=8):

    '''
    X = xarray dataset
    look_up = int, length of sequence of past observations to use for prediction
    look_ahead = int, length of sequence to predict
    '''

    self.X = X
    self.stations = list(X.keys())
    self.look_up = look_up
    self.look_ahead = look_ahead
    self.shape_0 = X.dims['datetime']
    self.shape_1 = len(self.stations)
    self.data_len = self.shape_0 // (self.look_up + self.look_ahead)
    self.df_distances = df_distances
    self.cut_off_km = cut_off_km
    self.n_neighbours = n_neighbours

  def __len__(self):
    return self.data_len * self.shape_1

  def __getitem__(self, i):

    def get_neighbours(ss_id):
      distances = self.df_distances[ss_id].sort_values().copy()
      distances = distances[distances > self.cut_off_km].copy()

      if len(distances) < self.n_neighbours:
        return np.random.choice(distances.index.values, self.n_neighbours, replace=True)

      else:
        return distances.index.values[list(range(0, len(distances), len(distances)//self.n_neighbours))[:self.n_neighbours]]


    ss_num = i // self.data_len
    ss_id = self.stations[ss_num]
    i = (i % self.shape_1) * (self.look_up + self.look_ahead)

    t = i + self.look_up

    neighbours = get_neighbours(ss_id)

    x_item = np.zeros((self.look_up, 5+self.n_neighbours))
    y_item = np.zeros((self.look_ahead, 1))

    x_item[:, 0], y_item = self.X[ss_id].data[t-self.look_up : t], self.X[ss_id].data[t : t+self.look_ahead]

    for i, ss in enumerate(neighbours):
      x_item[:, i+1] = self.X[ss].data[t-self.look_up : t]

    x_item[:, -4] = self.X.date_sin.data[t-self.look_up : t]
    x_item[:, -3] = self.X.date_cos.data[t-self.look_up : t]
    x_item[:, -2] = self.X.time_sin.data[t-self.look_up : t]
    x_item[:, -1] = self.X.time_sin.data[t-self.look_up : t]

    return torch.Tensor(x_item).float(), torch.Tensor(y_item).float()

In [ ]:
from torch.utils.data import DataLoader

def loaders(xr_test, xr_train, batch_size, dict_look_up, dict_look_ahead,
            df_metadata_train, df_metadata_test,
            cut_off_km=64, n_neighbours=8,
            df_distances_train=None, df_distances_test=None):

  if df_distances_train is None:
    df_distances_train = df_of_neighbours(df_metadata_train, df_metadata_train).copy()
  if df_distances_test is None:
    df_distances_test = df_of_neighbours(df_metadata_test, df_metadata_train).copy()

  max_len = len(xr_train['datetime'].data)

  look_up, look_ahead = get_seq_length(past_dict=dict_look_up, future_dict=dict_look_ahead, max_len=max_len)

  train_dataset = PVDataset(X=xr_train, df_distances=df_distances_train, look_up=look_up, look_ahead=look_ahead,
                            cut_off_km=cut_off_km, n_neighbours=n_neighbours)
  test_dataset = PVDataset(X=xr_test, df_distances=df_distances_test, look_up=look_up, look_ahead=look_ahead,
                            cut_off_km=cut_off_km, n_neighbours=n_neighbours)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  print("Expected number of steps is %d" %((train_dataset.__len__()  + batch_size - 1) // batch_size))

  return train_loader, test_loader, look_ahead

In [ ]:
class LSTM_model(nn.Module):

  def __init__(self, hidden_size,
               num_layers,
               out_features,
               input_size,
               dropout=0):

    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout
                        )

    self.fc = nn.Linear(in_features=hidden_size, out_features=out_features)

  def forward(self, x):

    batch_size = x.size(0)

    h_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)
    c_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)


    _, (h_out, _) = self.lstm(x, (h_t, c_t))

    h_out = h_out[0].view(-1, self.hidden_size)

    out = self.fc(h_out)

    return out

In [ ]:
def train_epoch(model, train_loader, optimizer, loss_function, print_every=500):
  loss_array = []
  model.train(True)
  running_loss = 0.

  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)

    loss = loss_function(output, y_batch.squeeze())
    running_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % print_every == 0:
      loss_array.append(running_loss/print_every)
      print("Batch %d, loss: %f" %(batch_index+1, running_loss/print_every))
      running_loss=0.

  loss_array.append(running_loss/((batch_index+1)%1000))

  print()
  return loss_array

In [ ]:
def validate_epoch(model, test_loader, loss_function):
  model.train(False)
  running_loss = 0.

  for batch_index, batch in enumerate(test_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      output = model(x_batch)
      loss = loss_function(output, y_batch.squeeze())
      running_loss += loss.item()

  avg_loss = running_loss / len(test_loader)

  print('Val loss: {0:.5f}'.format(avg_loss))
  print('_______________________________')
  print()

  return avg_loss

In [ ]:
best_params_df = pd.read_csv(path+'neighbours_best_params.csv')
best_params = best_params_df.loc[0].to_dict()

In [ ]:
n_layers = int(best_params['num_layers'])
hid_size = int(best_params['hidden_size'])
look_up = int(best_params['look_up'])
dropout = best_params['dropout']
cut_off_km = int(best_params['cut_off_km'])
n_neighbours = int(best_params['n_neighbours'])


data_params = {
'xr_test': xr_test,
'xr_train': xr_train,
'xr_test': xr_test,
'xr_train': xr_train,
'batch_size': 128,
'dict_look_up': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': look_up},
'dict_look_ahead': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 6},
'cut_off_km': cut_off_km,
'n_neighbours': n_neighbours,
'df_metadata_train': df_metadata_train,
'df_metadata_test': df_metadata_test,
'df_distances_train': df_distances_train,
'df_distances_test': df_distances_test
}

train_loader, test_loader, look_ahead = loaders(**data_params)

model_params = {
    'hidden_size': hid_size,
    'num_layers': n_layers,
    'out_features': look_ahead,
    'input_size': n_neighbours+1+4,
    'dropout': dropout
}

model = LSTM_model(**model_params)

model.to(device)

learning_rate = 0.001
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 6

train_loss = []
test_loss = []

best_loss = 1
best_epoch = 0
for epoch in range(num_epochs):
    print('Epoch:', epoch+1)
    train_loss.append(train_epoch(model=model, train_loader=train_loader,
                                optimizer=optimizer,
                                loss_function=loss_function,
                                print_every=300))
    test_loss.append(validate_epoch(model=model, test_loader=test_loader, loss_function=loss_function))
    if test_loss[-1] < best_loss:
        best_loss = test_loss[-1]
        best_epoch = epoch
    if epoch - best_epoch >= 2:
        break


torch.save(model.state_dict(), path+'neighbours_model_6ep')